In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import scipy
import scipy.stats as stats
from scipy.optimize import minimize
import threading
import random
import time
import multiprocessing as mp

In [ ]:
FOGGY = np.array([0.5, 0.25, 0.25])
SUNNY = np.array([0.5, 0, 0.5])
RAINY = np.array([0.25, 0.25, 0.5])

parameter = ["Foggy", "Sunny", "Rainy"]

p = np.array([FOGGY, SUNNY, RAINY])
print(p)

Prediction_Days = 25

In [ ]:
CurrentStateProb = [[0,0,0]]*Prediction_Days
CurrentStateProb[0] = [1,0,0]
#print(CurrentStateProb[1])

for i in range(1,Prediction_Days):
    Prob = np.matmul(CurrentStateProb[i-1], p)
    CurrentStateProb[i] = list(Prob)

#print(CurrentStateProb[24])

Foggy,Sunny,Rainy = [],[],[]
for i in range(0,Prediction_Days):
    Foggy.append(CurrentStateProb[i][0])
    Sunny.append(CurrentStateProb[i][1])
    Rainy.append(CurrentStateProb[i][2])

plt.plot(Foggy,'g')
plt.plot(Sunny,'r')
plt.plot(Rainy,'b')
plt.show()

In [ ]:
alpha_true = -2
beta_true = -5
time = list(range(1,11))

y_true = [alpha_true*time[i] + beta_true for i in range(0,10)]
sigma = 1
meas_err = np.random.normal(loc = 0, scale = sigma, size = len(time))
y = y_true + meas_err

In [ ]:
plt.plot(y,label='Error')
plt.plot(y_true,label='True')
#plt.plot(y_true,label='True')
plt.legend()
plt.show()

In [ ]:
def logp(theta,time,y,sigma=1):
    N = len(time)
    alpha,beta = theta[0],theta[1]
    model = [alpha*time[i] + beta for i in range(0,10)]
    resid = y - model
    #print(resid)
    log_likelihood = -N/2 * math.log(2*math.pi) - N*math.log(sigma) - 1/2*sum(resid**2)/sigma**2
    log_prior = 0 # log(1)
    log_posterior = log_likelihood + log_prior
    return log_posterior

In [ ]:
N = 30000
alpha_init = np.random.randint(-50,50)
beta_init = np.random.randint(-50,50)
theta = [alpha_init,beta_init]

In [ ]:
print(theta)
np.cov(theta,theta,theta)

In [ ]:
lp = logp(theta,time,y)

In [ ]:
def next_cov(t,xt,Ct,xtm,xtpm,xtp=0):
    d = 2
    sd = (2.4**2)/d
    e = 0.05
    Id = np.array([[1,0],[0,1]])
    p1 = (t-1)/t*Ct
    p2 = sd/t-(t*np.matmul(np.transpose(xtpm),xtpm)-(t+1)*np.matmul(np.transpose(xtm),xtm)+np.matmul(np.transpose(xt),xt)+e*Id)
    return p1+p2

In [ ]:
theta_best = theta
lp_max = lp
theta_new = [0]*2
accepts = 0
mcmc_chains = [[0,0]]*N
step = [.1,1]
time = list(range(1,11))
# Defining variables for Adaptive metropolis Hastings -


t = 0
xt = np.array(theta)
xtm = np.array(theta)
xtpm = np.array(theta)
cov = []
for i in range(N):   
    t += 1
    if (t == 1):
        cov = np.cov(theta,theta_new)
    step = np.array(cov[0,0],cov[1,1])
    #print(step)
    theta_new = list(stats.multivariate_normal.rvs(theta,cov))
    
    lp_new = logp(theta_new,time,y)
    lq = lp_new - lp
    lr = math.log(np.random.uniform(0,1))
    if (lr < lq):
        theta = theta_new
        lp = lp_new
        accepts += 1
        if lp > lp_max:
            theta_best = theta
            lp_max = lp
    mcmc_chains[i] = theta
    if (t > 1):
        xtpm = xtm
        xtm = xtpm + ((theta-xtpm)/t)
        cov = next_cov(t,xt,cov,xtm,xtpm)
        #print(cov)
    else:
        xtm = (xtpm+theta)/2 
    
print()

In [ ]:
print(cov)

In [ ]:
#print (mcmc_chains)
print(accepts/N*100)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(mcmc_chains,linewidth=0.25)
plt.show()

In [ ]:
alpha = [mcmc_chains[i][0] for i in range(len(mcmc_chains))]
beta = [mcmc_chains[i][1] for i in range(len(mcmc_chains))]

In [ ]:
def correlation(X, Y):
    pass

def plot_graph_beta(lagrange):
    cor,r,flag = [],None,True
    for lag in range(1,lagrange):
        X,Y = [],[]
        X = beta[:len(beta)-lag]
        Y = beta[lag:]
        #x = np.random.choice(X,size,replace=True)
        #y = x+np.random.choice(Y,size,replace=True)
        cor.append(np.corrcoef(X,Y,rowvar=False)[0][1])
        if flag and cor[lag-1] <= 0.05:
            flag = False
            r = lag
    plt.plot(range(1,lagrange),cor)
    plt.plot([r,r],[-0.2,1])
    title = 'Lag '+str(lag)
    plt.title(title)
    plt.xlabel('Lag')
    plt.ylabel('correlation')
    plt.show()
    return r

def plot_graph_alpha(lagrange):
    cor,r,flag = [],None,True
    for lag in range(1,lagrange):
        X,Y = [],[]
        X = alpha[:len(alpha)-lag]
        Y = alpha[lag:]
        #x = np.random.choice(X,size,replace=True)
        #y = x+np.random.choice(Y,size,replace=True)
        cor.append(np.corrcoef(X,Y,rowvar=False)[0][1])
        if flag and cor[lag-1] <= 0.05:
            flag = False
            r = lag
    plt.plot(range(1,lagrange),cor)
    plt.plot([r,r],[-0.2,1])
    title = 'Lag '+str(lag)
    plt.title(title)
    plt.xlabel('Lag')
    plt.ylabel('correlation')
    plt.show()
    return r

In [ ]:
lag1 = plot_graph_beta(1500)
lag2= plot_graph_alpha(1500)

In [ ]:
print(lag1)
print(lag2)
alpha1,beta1 = [],[]
for i in range(0,len(mcmc_chains),40):
    alpha1.append(mcmc_chains[i][0])
    beta1.append(mcmc_chains[i][1])
print('Alpha estimate',np.mean(alpha1))
print('Beta estimate',np.mean(beta1))

In [ ]:
data = pd.read_csv('observations.txt',sep="\t")
t = data['time']
observations = data['observations']

plt.plot(t,observations,'b.')
plt.xlabel('Time')
plt.ylabel('Observations')
plt.show()

In [ ]:
def model(parm,t):
    model_p = len(parm)
    alpha = parm[0]
    beta = parm[1]
    y_mod = alpha*t + beta
    return [list(y_mod),model_p]

def fn(parameters, t, obs):
    alpha = parameters[0]
    beta = parameters[1]
    data = alpha*t + beta
    resid = obs - data
    rmse = math.sqrt(np.mean(resid**2))
    return rmse

paramater_guess = [0.5,2]
result = minimize(fn, paramater_guess, args=(t,observations), method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
start_alpha = result.x[0]
start_beta = result.x[1]
parameter = [start_alpha,start_beta]
y_obs = model(parameter,t)
res = observations - y_obs[0]
start_sigma = np.std(res)

In [ ]:
plt.plot(res)
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.show()

In [ ]:
bound_lower = [-1,-1,0]
bound_upper = [1,3,1]
parnames = ['alpha','beta','sigma']
model_p = 2

In [ ]:
step = [0.001, 0.01, 0.01]
NI = 1e3

In [ ]:
st = [[1/2,1/6,1/6,1/6,0,0],[1/2,1/6,1/6,1/6,0,0],[1/2,1/6,1/6,1/6,0,0],[0,0,1/6,1/6,1/2,1/6],[0,0,1/6,1/6,1/2,1/6],[0,0,1/6,1/6,1/2,1/6]]
st = np.array(st)
st = np.transpose(st)

print(st)
multi = np.array([1,0,0,0,0,0])
for i in range(100):
    multi = np.dot(st,multi)
print(multi)
print(sum(multi[1:4])/sum(multi[:4]))
print(0.25/(0.25+0.08333333))
print()
d,v = np.linalg.eig(st)
print(d)
print(v[:,0]/sum(v[:,0]))

In [ ]:
def cm(n):
    total = 0
    for i in range(n):
        total += 2**i / (5**(2+i))
    return total

In [ ]:
#calculating expected value -
ex = 0
for i in range(1,5):
    print(i*cm(i))
    ex += i*cm(i)

In [ ]:
print(ex)

In [ ]:
def countWays(arr, N):
    count = [0]*(N+1)
    count[0] = 1
    for i in range(N+1):
        for j in range(len(arr)):
            if i >= arr[j]:
                count[i] += count[i - arr[j]]
    return count[N]

In [ ]:
countWays(range(1,6),5)

In [ ]:
def loop():
    arr = np.array(range(1,6))
    sum1,sum2 = 0,0
    sum1 += np.random.choice(arr)
    sum2 += np.random.choice(arr)
    while (sum1 != sum2):
        if sum1 < sum2:
            sum1 += np.random.choice(arr)
        else:
            sum2 += np.random.choice(arr)
    return sum1

In [ ]:
temp = []
for i in range(1000000):
    temp.append(loop())
print(sum(temp)/len(temp))

In [ ]:
st = np.array([[1,0,0,0,0,0,0],[1/5,1/5,1/5,1/5,1/5,0,0],[1/5,2/5,1/5,1/5,0,0,0],[1/5,2/5,2/5,0,0,0,0],[1/5,2/5,1/5,1/5,0,0,0],[1/5,1/5,1/5,1/5,1/5,0,0],[0,1/5,1/5,1/5,1/5,1/5,0]])
st = st[1:,1:]
N = np.linalg.inv(np.eye(6) - st)
print(np.matmul(N,np.ones(6)))

In [ ]:
size = 30
st = []
for i in range(0,size+1):
    temp = np.zeros(size+1)
    if i == 0:
        temp[size-1],temp[1] = 1/2,1/2
    elif i == size-1:
        temp[i-1],temp[0] = 1/2,1/2
    elif i == size:
        temp[0],temp[size-1] = 1/2,1/2
    else:
        temp[i+1],temp[i-1] = 1/2,1/2
    st.append(temp)

print(len(st[0]))

In [ ]:
st = np.array(st)
st = np.transpose(st)

#print(st)
multi = np.zeros(size+1)
multi[0] = 1
for i in range(100000):
    multi = np.dot(st,multi)
print(multi)
print()
d,v = np.linalg.eig(st)
print(d)

print(v[:,4]/sum(v[:,4]))

# Multi-threading

In [2]:
output1 = mp.SimpleQueue()
def rand_string(length,output):
    output.put(1)
def main():
    p = mp.Process(target=rand_string, args=(5,output1))
    # Run processes
    temp = p.start()
    # Exit the completed processes
    print(p)
    p.join()
    print(p)
    # Get process results from the output queue

In [3]:
if __name__ == '__main__':
    mp.freeze_support()
    main()

<Process(Process-1, started)>
<Process(Process-1, stopped[1])>


In [4]:
output1.put(5)
print(output1.empty())

False
